In [1]:
from helper import *
from helper2 import *

In [2]:
import pandas as pd
import os
notebook_path = os.path.abspath('')
print(notebook_path)

/home/dimitris/Documents/github/EPF_ABA23


In [32]:
path_ = './dataset_management/data/clean/clean_lagged.csv'

In [36]:
df =read_our_data('clean_lagged.csv')

In [37]:
df.set_index('Timestamp')

,DK_1,SE_4,DK_1_Exports_Volume,SE_4_Exports_Volume,Day_Ahead_Prices,Forecasted Load,Actual Load,Solar,TTF=F_Price,CFI2Z3_Price,...,Hour,Week,business,Wind Total,Day_Ahead_Prices-lag24,Day_Ahead_Prices-lag36,Day_Ahead_Prices-lag48,Day_Ahead_Prices-lag72,Day_Ahead_Prices-lag168,Day_Ahead_Prices-lag-168
Timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-09 00:00:00,1.0,0.0,129.0,560.0,27.17,1309.0,1309.0,0.0,19.434999,107.00,...,0,2,True,393.0,25.03,28.96,27.58,29.45,18.12,-9.61
2018-01-09 01:00:00,84.0,0.0,2.0,874.0,22.10,1233.0,1221.0,0.0,19.434999,107.00,...,1,2,True,439.0,23.96,28.27,25.90,28.55,14.99,-6.62
2018-01-09 02:00:00,73.0,0.0,0.0,679.0,9.79,1186.0,1155.0,0.0,19.434999,107.00,...,2,2,True,441.0,23.92,28.29,25.89,28.18,17.79,-2.33
2018-01-09 03:00:00,141.0,0.0,0.0,714.0,4.99,1175.0,1139.0,0.0,19.434999,107.00,...,3,2,True,457.0,25.48,28.62,26.16,28.00,24.51,-4.92
2018-01-09 04:00:00,163.0,0.0,0.0,643.0,9.24,1184.0,1125.0,0.0,19.434999,107.00,...,4,2,True,480.0,26.12,29.38,26.60,28.15,17.82,-0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-11 19:00:00,0.0,1191.0,122.0,0.0,165.38,1566.0,1579.0,2.0,43.692001,194.15,...,19,15,True,169.0,36.12,69.43,133.46,139.77,177.32,104.47
2023-04-11 20:00:00,0.0,1133.0,232.0,0.0,141.39,1473.0,1520.0,0.0,43.692001,194.15,...,20,15,True,171.0,26.18,48.13,126.39,137.31,155.56,84.65
2023-04-11 21:00:00,0.0,1174.0,355.0,0.0,130.80,1394.0,1412.0,0.0,43.692001,194.15,...,21,15,True,162.0,14.98,27.45,115.00,134.90,141.19,69.67


In [38]:
#df.info()
df.columns.values

array(['Timestamp', 'DK_1', 'SE_4', 'DK_1_Exports_Volume',
       'SE_4_Exports_Volume', 'Day_Ahead_Prices', 'Forecasted Load',
       'Actual Load', 'Solar', 'TTF=F_Price', 'CFI2Z3_Price',
       'CO2_prices_Price', 'DE_LU_corr', 'DE_LU_Exports_Volume_corr',
       'Year', 'Quarter', 'Month', 'Date', 'Day', 'Weekday', 'Hour',
       'Week', 'business', 'Wind Total', 'Day_Ahead_Prices-lag24',
       'Day_Ahead_Prices-lag36', 'Day_Ahead_Prices-lag48',
       'Day_Ahead_Prices-lag72', 'Day_Ahead_Prices-lag168',
       'Day_Ahead_Prices-lag-168'], dtype=object)

In [43]:
cols = df.columns.values.tolist()

In [44]:
type(cols)

list

In [45]:
cols

['Timestamp',
 'DK_1',
 'SE_4',
 'DK_1_Exports_Volume',
 'SE_4_Exports_Volume',
 'Day_Ahead_Prices',
 'Forecasted Load',
 'Actual Load',
 'Solar',
 'TTF=F_Price',
 'CFI2Z3_Price',
 'CO2_prices_Price',
 'DE_LU_corr',
 'DE_LU_Exports_Volume_corr',
 'Year',
 'Quarter',
 'Month',
 'Date',
 'Day',
 'Weekday',
 'Hour',
 'Week',
 'business',
 'Wind Total',
 'Day_Ahead_Prices-lag24',
 'Day_Ahead_Prices-lag36',
 'Day_Ahead_Prices-lag48',
 'Day_Ahead_Prices-lag72',
 'Day_Ahead_Prices-lag168',
 'Day_Ahead_Prices-lag-168']

In [50]:
features = cols.remove(['Timestamp', 'Day_Ahead_Prices'])

ValueError: list.remove(x): x not in list

In [47]:
target = 'Day_Ahead_Prices'

In [48]:
# setting up splits
k_folds = 3
train_start = pd.to_datetime(['2018-01-01', '2021-01-01', '2021-07-01'], format = '%Y-%m-%d')

In [27]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from operator import itemgetter

In [28]:
import statsmodels.api as sm

In [32]:
# this is to call the linear regression importance feature
lr_imp = lin_reg_feat_importance(x_train, y_train, x_test, y_test)

NameError: name 'lin_reg_feat_importance' is not defined

In [3]:
# plot
var_imp_plot(lr_imp, 'Linear regression feature importance')

NameError: name 'lr_imp' is not defined

In [4]:
# vlepoume
# def feature_random_forest(x_train, y_train):
#     rf_selector = RandomForestClassifier(n_estimators=100)
#     fit = rf_selector.fit(x_train, y_train)

#     cols = pd.DataFrame(x_train.columns)
#     ranking = pd.DataFrame(fit.feature_importances_)
#     rfe_selector = pd.concat([cols,ranking], axis=1)
#     rfe_selector.columns = ['Feature','Ranking']  # name the dataframe columns

#     rfe_selector = rfe_selector.nlargest(41,'Ranking').reset_index().drop('index',axis=1)
#     print("Num Features: %s" % (fit.n_features_))
#     print(rfe_selector)